# 3-qubit teleportation with post-selection
Here we demonstrate teleportation without active feed-forward.  Instead we keep
only the measurement branch where the X-basis measurement on `q1` yields `s=0`,
producing a high-fidelity Bell pair between `q0` and `q2` at the cost of a
post-selection filter.


In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
from qiskit_aer import AerSimulator

## Build circuits and capture conditional states
The next cell prepares both the original Bell pair and the teleportation circuit
with conditional density-matrix saves so that each measurement outcome can be
inspected separately.


In [2]:
# =====================================================
# 1. Original entangled state on qubits (0,1)
# =====================================================

orig_circ = QuantumCircuit(3)
orig_circ.h(0)
orig_circ.cx(0, 1)

orig_circ.save_density_matrix(label="rho_orig")

sim_dm = AerSimulator(method="density_matrix")
orig_compiled = transpile(orig_circ, sim_dm)
orig_result = sim_dm.run(orig_compiled).result()

rho_full_orig = DensityMatrix(orig_result.data(0)["rho_orig"])
rho_01 = partial_trace(rho_full_orig, [2])  # ideal reference state


# =====================================================
# 2. Teleportation circuit WITH *POSTSELECTION*, no feed-forward
# =====================================================

tele_circ_dm = QuantumCircuit(3, 1)

# Prepare entangled state on (0,1)
tele_circ_dm.h(0)
tele_circ_dm.cx(0, 1)

# Prepare q2 in |+>
tele_circ_dm.h(2)

# CZ between q1 and q2
tele_circ_dm.cz(1, 2)

# Measure q1 in X basis
tele_circ_dm.h(1)
tele_circ_dm.measure(1, 0)   # store s in classical bit

# NO feed-forward X gate
# tele_circ_dm.x(2).c_if(... )   <-- REMOVED

# Always apply final H on qubit 2
tele_circ_dm.h(2)

# Save conditional density matrices, one per classical outcome
tele_circ_dm.save_density_matrix(label="rho_final", conditional=True)

tele_compiled = transpile(tele_circ_dm, sim_dm)
tele_result = sim_dm.run(tele_compiled).result()


# =====================================================
# 3. POST-SELECTION: keep only measurement s=0
# =====================================================

rho_conds = tele_result.data(0)["rho_final"]
print("Conditional keys:", rho_conds.keys())  # shows ['0x1', '0x0']

# Postselect on s = 0  → key '0x0'
rho_post = rho_conds['0x0']

# Reduce to qubits (0,2)
rho_02 = partial_trace(rho_post, [1])

print("Post-selected density matrix for (0,2):")
print(rho_02)

Conditional keys: dict_keys(['0x0', '0x1'])
Post-selected density matrix for (0,2):
DensityMatrix([[ 5.00000000e-01+9.52769318e-34j,
                -1.84889275e-32-3.06161700e-17j,
                -1.84889275e-32-3.06161700e-17j,
                 5.00000000e-01+6.12323400e-17j],
               [-1.84889275e-32+3.06161700e-17j,
                 1.84889275e-32-9.52769318e-34j,
                 1.84889275e-32-3.17589773e-34j,
                -1.84889275e-32+3.06161700e-17j],
               [-1.84889275e-32+3.06161700e-17j,
                 1.84889275e-32+3.17589773e-34j,
                 1.84889275e-32-9.52769318e-34j,
                -1.84889275e-32+3.06161700e-17j],
               [ 5.00000000e-01-6.12323400e-17j,
                -1.84889275e-32-3.06161700e-17j,
                -1.84889275e-32-3.06161700e-17j,
                 5.00000000e-01+9.52769318e-34j]],
              dims=(2, 2))


## Quantify teleportation quality
Finally we reduce the post-selected state to qubits `(0,2)` and compare it with
our source Bell pair via the state fidelity metric.


In [ ]:
# =====================================================
# 3. Fidelity between original (0,1) and teleported (0,2)
# =====================================================

F = state_fidelity(rho_01, rho_02)
print("Fidelity between original (q0,q1) and teleported (q0,q2):", F)

Fidelity between original (q0,q1) and teleported (q0,q2): 1.0000000000000004
